In [22]:
import random
import numpy as np
from matplotlib import pyplot as plt

In [23]:
def is_satisfied(formula, valuation):
    for clause in formula:
        clause_satisfied = False
        for literal in clause:
            variable = abs(literal)
            is_negated = False
            if literal < 0:
                is_negated = True
            if is_negated:
                value = not valuation[variable]
            else:
                value = valuation[variable]
            if value:
                clause_satisfied=True
                break
        
        if not clause_satisfied:
            return False
    return True

In [24]:
def count_satisfied_formulas(formulas, valuation):
    count = 0
    for formula in formulas:
        if is_satisfied(formula, valuation):
            count += 1
    return count

In [25]:
def generate_random_valuation(variables):
    return {var:random.choice([True,False]) for var in variables}

In [26]:
class Individual:
    def __init__(self,variables,formulas):
        self.code=generate_random_valuation(variables)
        self.fitness=self.calc_fitness(formulas)
    
    def calc_fitness(self,formulas):
        return -count_satisfied_formulas(formulas,self.code)
    
    def __lt__(self, other):
        return self.fitness < other.fitness

In [27]:
def selection(population, tournament_size, forbidden):
    allowed = list(set(range(len(population))).difference({forbidden}))
    chosen = random.sample(allowed, tournament_size)
    max_fitness = float('-inf')
    best_idx = -1
    for i in chosen:
        if population[i].fitness > max_fitness:
            max_fitness = population[i].fitness
            best_idx = i
    return best_idx

In [28]:
def crossover(parent1, parent2, child1, child2):
    pos = random.randrange(1, len(parent1.code)-1)
    
    child1.code={var:parent1.code[var] for var in parent1.code}
    child2.code={var:parent2.code[var] for var in parent2.code}
    
    for i,var in enumerate(parent1.code):
        if i>=pos:
            child1.code[var]=parent2.code[var]
            child2.code[var]=parent1.code[var]

In [29]:
def mutation(child, mutation_prob):
    for var in child.code:
        if random.random()<mutation_prob:
            child.code[var]=not child.code[var]

In [30]:
def ga(pop_size, num_iters, tournament_size, mutation_prob, use_elitism, elitism_size):
    if use_elitism and (pop_size - elitism_size) % 2 == 1:
        elitism_size += 1

    population = [Individual(variables,formulas) for _ in range(pop_size)]
    new_population = [Individual(variables,formulas) for _ in range(pop_size)]
    for i in range(num_iters):
        if use_elitism:
            population.sort(key=lambda x: x.fitness, reverse=True)
            
            new_population[:elitism_size] = population[:elitism_size]
            
        for j in range(elitism_size, pop_size, 2):
            parent1_idx = selection(population, tournament_size, None)
            parent2_idx = selection(population, tournament_size, parent1_idx)
            
            crossover(population[parent1_idx],
                      population[parent2_idx],
                      new_population[j],
                      new_population[j+1])
            
            mutation(new_population[j], mutation_prob)
            mutation(new_population[j+1], mutation_prob)
            
            new_population[j].fitness = new_population[j].calc_fitness( formulas)
            new_population[j+1].fitness = new_population[j+1].calc_fitness( formulas)

        population[:] = new_population[:]
        
    best_individual = max(population, key=lambda x: x.fitness)
    #print("Minimum number of satisfiable formulas:", -best_individual.fitness)
    #print("Valuation for minimum number of satisfiable formulas:",best_individual.code)
    return -best_individual.fitness,best_individual.code